In [2]:
import ccxt
import pandas as pd
import numpy as np

class LiquidityDepthAnalyzer:
    """
    Module 2: Market Liquidity & Execution Risk Analysis.
    Evaluates the systemic capacity of the exchange order book to absorb 
    large-scale liquidation events during high-volatility regimes.
    """
    def __init__(self, symbol='BTC/USDT'):
        self.exchange = ccxt.binance()
        self.symbol = symbol

    def fetch_l2_order_book(self):
        """
        Retrieves real-time Layer-2 (L2) Order Book data.
        Captures the top 100 price levels of the 'Bid' side to model 
        forced-selling scenarios.
        """
        print(f"[*] Extracting L2 Order Book depth for {self.symbol}...")
        # Fetching snapshots to simulate liquidity availability
        order_book = self.exchange.fetch_order_book(self.symbol, limit=100)
        return order_book

    def evaluate_slippage_risk(self, notion_size_btc=1000):
        """
        Quantifies 'Price Impact' and 'Execution Slippage' for a hypothetical 
        liquidity drain of 1,000 BTC.
        
        This models the secondary crash risk if the Protocol's safety 
        buffers were bypassed.
        """
        ob = self.fetch_l2_order_book()
        bids = ob['bids']  # Liquidity providers (Bids) protecting the downside
        
        accumulated_volume = 0
        weighted_cost = 0
        mid_market_price = bids[0][0] # Initial best bid price
        
        for price, amount in bids:
            if accumulated_volume + amount >= notion_size_btc:
                remaining_fill = notion_size_btc - accumulated_volume
                weighted_cost += remaining_fill * price
                accumulated_volume += remaining_fill
                break
            else:
                accumulated_volume += amount
                weighted_cost += amount * price
        
        # Calculate Volume Weighted Average Price (VWAP) for the liquidation
        execution_vwap = weighted_cost / notion_size_btc
        # Quantify slippage as a percentage deviation from mid-market
        slippage_pct = (mid_market_price - execution_vwap) / mid_market_price
        
        print(f"\n[!] Stress Test Result: Counterfactual Liquidation Scenario")
        print(f"    - Simulated Exit Volume : {notion_size_btc} BTC")
        print(f"    - Execution VWAP       : ${execution_vwap:,.2f}")
        print(f"    - Market Price Impact  : {slippage_pct:.4%} (Slippage)")
        
        # Risk assessment based on slippage thresholds
        if slippage_pct > 0.05:
            print("    >>> WARNING: High systemic risk - Depth insufficient for exit.")
        else:
            print("    >>> STATUS: Market resilience confirmed - Adequate depth.")
            
        return slippage_pct

if __name__ == "__main__":
    analyzer = LiquidityDepthAnalyzer()
    # Stress test: Simulating a 1,000 BTC liquidation event
    analyzer.evaluate_slippage_risk(notion_size_btc=1000)

[*] Extracting L2 Order Book depth for BTC/USDT...

[!] Stress Test Result: Counterfactual Liquidation Scenario
    - Simulated Exit Volume : 1000 BTC
    - Execution VWAP       : $1,021.73
    - Market Price Impact  : 98.8380% (Slippage)
    >>> WARNING: High systemic risk - Depth insufficient for exit.
